This notebook follows from the tutorials of CHGNet to relax LLZO-Li slabs that have been made in `llzo_li_balanced_sliced` directory.


LLZO‖Li Interface Relaxation Notebook
========================================

Each notebook handles only one structure.
1. Purpose: (This keeps updating)
- Relax a single LLZO‖Li (in this notebook LLZO_001_Zr_code93_sto__Li_110_slab_heavy) heterostructure using CHGNet
- Perform multi-stage optimization (in this notebook, CG → FIRE)
- Freeze bulk-like regions (15 Å at both ends)
- after all this, Relax lattice vectors to relieve interface strain

2. This notebook handles:
- Structure: LLZO_110_Li_order17_off__Li_100_slab_heavy
- Initial lattice height: 86.02 Å
- Number of atoms: 738

3. Method:
- CHGNet (v0.4.0) + ASE interface
- Stage 1: SciPyFminCG (no cell relaxation) → fmax target ~0.15 eV/Å
- Stage 2: FIRE (with optional cell relaxation) → fmax target ~0.05 eV/Å
- FrechetCellFilter used for combined force + stress minimization

4. Constraints:
- LLZO base: frozen bottom 14.25 Å
- Li top: frozen top 14.25 Å
- Only interfacial region relaxed
- Cell relaxation via `relax_cell=True` and `relax_cell_atoms="unconstrained"`

5. Outputs: (This will be decided later)
- relaxed_[structure_name].cif
- relaxed_[structure_name].traj
- (Optional) relaxation_log.pkl with energies, forces

6. Visual checks:
- Compare pre- and post-relaxation structures
- Ensure no Li diffusion into LLZO (via z-analysis)
- Confirm convergence (fmax < 0.05 eV/Å)

Author: Mehul Darak

Date: 15-07-2025


In [1]:
structure_name = "LLZO_011_La_code71_sto__Li_100_slab_heavy"

In [2]:
from pymatgen.core import Structure
import os

# Load structure
structure_path = (f"/home/mehuldarak/summer/llzo_li_balanced_sliced/{structure_name}.cif")  # replace with your file
structure = Structure.from_file(structure_path)

# Extract info
structure_name = os.path.basename(structure_path).replace(".cif", "")
lattice_height = structure.lattice.c
num_atoms = len(structure)

# Print output
print(f"- Structure: {structure_name}")
print(f"- Initial lattice height: {lattice_height:.2f} Å")
print(f"- Number of atoms: {num_atoms}")


- Structure: LLZO_011_La_code71_sto__Li_100_slab_heavy
- Initial lattice height: 85.95 Å
- Number of atoms: 756


In [3]:
from pymatgen.core import Structure
import numpy as np

s = Structure.from_file(f"/home/mehuldarak/summer/llzo_li_balanced_sliced/{structure_name}.cif")

# Get all atoms
z_coords = np.array([site.z for site in s.sites])
species = np.array([site.species_string for site in s.sites])

# Estimate LLZO top (non-Li atoms)
llzo_z = z_coords[species != "Li"]
llzo_top = llzo_z.max()

# Now isolate Li slab: Li atoms ABOVE LLZO
li_slab_z = np.array([site.z for site in s.sites if site.species_string == "Li" and site.z > llzo_top])

print(f"Li slab thickness: {li_slab_z.ptp():.2f} Å")
print(f"Lowest Li slab atom: {li_slab_z.min():.2f} Å")
print(f"LLZO top z: {llzo_top:.2f} Å")
print(f"Li penetration into LLZO: {llzo_top - li_slab_z.min():.2f} Å")


Li slab thickness: 24.82 Å
Lowest Li slab atom: 46.13 Å
LLZO top z: 42.13 Å
Li penetration into LLZO: -4.00 Å


In [4]:
from pymatgen.core import Structure
import numpy as np

# Load structure
structure = Structure.from_file(f"/home/mehuldarak/summer/llzo_li_balanced_sliced/{structure_name}.cif")

# Get z-coordinates and element types
z_coords = np.array([site.z for site in structure.sites])
species = np.array([site.species_string for site in structure.sites])

# LLZO: non-Li atoms (La, Zr, O)
llzo_z = z_coords[species != "Li"]
llzo_top = llzo_z.max()
llzo_bottom = llzo_z.min()
llzo_thickness = llzo_top - llzo_bottom

# Li slab: Li atoms ABOVE LLZO (i.e. in metallic Li layer)
li_slab_z = np.array([
    site.z for site in structure.sites
    if site.species_string == "Li" and site.z > llzo_top
])
li_thickness = li_slab_z.ptp() if len(li_slab_z) > 0 else 0
li_bottom = li_slab_z.min() if len(li_slab_z) > 0 else None

# Penetration check
penetration = llzo_top - li_bottom if li_bottom is not None else 0

# Report
print(f"LLZO slab thickness: {llzo_thickness:.2f} Å")
print(f"Li slab thickness:   {li_thickness:.2f} Å")
print(f"LLZO top z:          {llzo_top:.2f} Å")
print(f"Lowest Li atom z:    {li_bottom:.2f} Å" if li_bottom else "No Li slab atoms found")
print(f"Li penetration into LLZO: {penetration:.2f} Å")


LLZO slab thickness: 27.13 Å
Li slab thickness:   24.82 Å
LLZO top z:          42.13 Å
Lowest Li atom z:    46.13 Å
Li penetration into LLZO: -4.00 Å


In [5]:
from pymatgen.io.ase import AseAtomsAdaptor
from ase.constraints import FixAtoms
from chgnet.model.dynamics import CHGNetCalculator, StructOptimizer
from ase.io import read, write
import numpy as np

# --- Load structure ---
structure = read(f"/home/mehuldarak/summer/llzo_li_balanced_sliced/{structure_name}.cif")

# --- Get z coordinates ---
z_coords = structure.get_positions()[:, 2]
z_min, z_max = z_coords.min(), z_coords.max()

# --- Define freeze zones ---
freeze_thickness_llzo = 0.75 * (llzo_thickness)  # in Å
freeze_thickness_li = 0.75 * (llzo_thickness)  # in Å
llzo_z_threshold = z_min + freeze_thickness_llzo
li_z_threshold = z_max - freeze_thickness_li

# --- Freeze LLZO base and Li top ---
freeze_mask = (z_coords < llzo_z_threshold) | (z_coords > li_z_threshold)
structure.set_constraint(FixAtoms(mask=freeze_mask))
print(f"Freezing {np.sum(freeze_mask)} atoms out of {len(structure)}")

# --- Attach CHGNet calculator ---
calc = CHGNetCalculator(use_device="cuda")
structure.set_calculator(calc)

Freezing 574 atoms out of 756
CHGNet v0.3.0 initialized with 412,525 parameters
CHGNet will run on cuda


/tmp/ipykernel_177739/3870098642.py:27: FutureWarning: Please use atoms.calc = calc
  structure.set_calculator(calc)


In [6]:
# Stage 1: CG
opt1 = StructOptimizer(model=calc, optimizer_class="SciPyFminCG", use_device="cuda")
result1 = opt1.relax(structure, fmax=0.15, steps=300, relax_cell=False, verbose=True)

             Step     Time          Energy          fmax
SciPyFminCG:    0 21:26:48    -2038.897593       14.964495
SciPyFminCG:    1 21:26:50    -2081.453127       20.672494
SciPyFminCG:    2 21:26:52    -2087.338469        9.910303
SciPyFminCG:    3 21:26:53    -2094.550770        8.691725
SciPyFminCG:    4 21:26:55    -2098.123936        3.667784
SciPyFminCG:    5 21:26:56    -2100.759650        2.162472
SciPyFminCG:    6 21:26:59    -2102.608237        3.051497
SciPyFminCG:    7 21:27:01    -2113.511765       11.739069
SciPyFminCG:    8 21:27:03    -2119.737047       16.209556
SciPyFminCG:    9 21:27:06    -2147.378975       11.918743
SciPyFminCG:   10 21:27:06    -2177.471867       14.431657
SciPyFminCG:   11 21:27:09    -2254.994823       18.144662
SciPyFminCG:   12 21:27:09    -2266.237930        7.519126
SciPyFminCG:   13 21:27:11    -2281.776083        2.935597
SciPyFminCG:   14 21:27:13    -2285.563740        2.446669
SciPyFminCG:   15 21:27:15    -2288.982977        2.210264

In [7]:
# Convert back, assign calculator + constraint
structure_1 = AseAtomsAdaptor.get_atoms(result1["final_structure"])
structure_1.set_calculator(calc)
structure_1.set_constraint(FixAtoms(mask=freeze_mask))

# Stage 2: FIRE
opt2 = StructOptimizer(model=calc, optimizer_class="FIRE", use_device="cuda")
result2 = opt2.relax(structure_1, fmax=0.05, steps=400, relax_cell=False, verbose=True)

/tmp/ipykernel_177739/3154571052.py:3: FutureWarning: Please use atoms.calc = calc
  structure_1.set_calculator(calc)


      Step     Time          Energy          fmax
FIRE:    0 21:30:25    -2338.234051        0.143408
FIRE:    1 21:30:26    -2338.236394        0.091109
FIRE:    2 21:30:27    -2338.239278        0.103939
FIRE:    3 21:30:28    -2338.241621        0.123731
FIRE:    4 21:30:29    -2338.243423        0.080435
FIRE:    5 21:30:30    -2338.247028        0.098452
FIRE:    6 21:30:31    -2338.250273        0.078605
FIRE:    7 21:30:31    -2338.253517        0.080900
FIRE:    8 21:30:32    -2338.257483        0.098804
FIRE:    9 21:30:33    -2338.262169        0.086986
FIRE:   10 21:30:34    -2338.267035        0.096682
FIRE:   11 21:30:35    -2338.272803        0.106916
FIRE:   12 21:30:36    -2338.279833        0.114806
FIRE:   13 21:30:37    -2338.288665        0.109063
FIRE:   14 21:30:38    -2338.298578        0.106991
FIRE:   15 21:30:39    -2338.307230        0.226107
FIRE:   16 21:30:40    -2338.307951        0.567607
FIRE:   17 21:30:40    -2338.319126        0.088603
FIRE:   18 21:

In [8]:
from pymatgen.io.ase import AseAtomsAdaptor
from ase.io import write

# Extract final structure from result3 (FIRE)
final_structure_pmg = result2["final_structure"]  # assuming result2 = FIRE
final_structure_ase = AseAtomsAdaptor.get_atoms(final_structure_pmg)

# Save as CIF and ASE trajectory
write(f"relaxed_{structure_name}.cif", final_structure_ase)
write(f"relaxed_{structure_name}.traj", final_structure_ase)

print("✅ Final structure saved successfully.")

✅ Final structure saved successfully.


In [9]:
from pymatgen.core import Structure
import numpy as np

s = Structure.from_file(f"/home/mehuldarak/summer/relax_final/{structure_name}/relaxed_{structure_name}.cif")

# Get all atoms
z_coords = np.array([site.z for site in s.sites])
species = np.array([site.species_string for site in s.sites])

# Estimate LLZO top (non-Li atoms)
llzo_z = z_coords[species != "Li"]
llzo_top = llzo_z.max()

# Now isolate Li slab: Li atoms ABOVE LLZO
li_slab_z = np.array([site.z for site in s.sites if site.species_string == "Li" and site.z > llzo_top])

print(f"Li slab thickness: {li_slab_z.ptp():.2f} Å")
print(f"Lowest Li slab atom: {li_slab_z.min():.2f} Å")
print(f"LLZO top z: {llzo_top:.2f} Å")
print(f"Li penetration into LLZO: {llzo_top - li_slab_z.min():.2f} Å")


Li slab thickness: 29.13 Å
Lowest Li slab atom: 41.82 Å
LLZO top z: 41.79 Å
Li penetration into LLZO: -0.02 Å


In [10]:
import os
from chgnet.model import StructOptimizer
from pymatgen.core import Structure
from chgnet.model.dynamics import CHGNetCalculator

structure_path = f"/home/mehuldarak/summer/relax_final/{structure_name}/relaxed_{structure_name}.cif"
structure = Structure.from_file(structure_path)

# Output path
output_dir = f"/home/mehuldarak/summer/relax_final"
os.makedirs(output_dir, exist_ok=True)
output_path = os.path.join(output_dir, f"cellrelaxed_{structure_name}.cif")

# Run CHGNet relaxation
opt1 = StructOptimizer(model=calc, optimizer_class="SciPyFminCG", use_device="cuda")
result = opt1.relax(
    structure,
    fmax=0.15,           # You can adjust depending on accuracy/speed tradeoff
    steps=400,
    relax_cell=True,
    verbose=True
)

             Step     Time          Energy          fmax
SciPyFminCG:    0 21:31:01    -2338.328679       17.010397
SciPyFminCG:    1 21:31:05    -2386.141222       18.743648
SciPyFminCG:    2 21:31:07    -2394.817829       14.919392
SciPyFminCG:    3 21:31:08    -2416.539989        8.393922
SciPyFminCG:    4 21:31:09    -2424.589165       10.090632
SciPyFminCG:    5 21:31:10    -2432.339676        8.785642
SciPyFminCG:    6 21:31:11    -2440.057563        3.732340
SciPyFminCG:    7 21:31:15    -2499.685850       23.046222
SciPyFminCG:    8 21:31:17    -2510.750155       19.301120
SciPyFminCG:    9 21:31:19    -2560.121813        6.549053
SciPyFminCG:   10 21:31:21    -2596.078777        3.839109
SciPyFminCG:   11 21:31:23    -2604.138047        2.028886
SciPyFminCG:   12 21:31:24    -2609.715351        1.261736
SciPyFminCG:   13 21:31:26    -2615.853035        1.438595
SciPyFminCG:   14 21:31:28    -2621.133116        1.841346
SciPyFminCG:   15 21:31:30    -2624.790456        2.604786

In [11]:
# Convert back, assign calculator + constraint
structure_1 = AseAtomsAdaptor.get_atoms(result["final_structure"])
structure_1.set_calculator(calc)
structure_1.set_constraint(FixAtoms(mask=freeze_mask))

# Stage 2: FIRE
opt2 = StructOptimizer(model=calc, optimizer_class="FIRE", use_device="cuda")
result2 = opt2.relax(structure_1, fmax=0.05, steps=400, relax_cell=True, verbose=True)

/tmp/ipykernel_177739/1077038773.py:3: FutureWarning: Please use atoms.calc = calc
  structure_1.set_calculator(calc)


      Step     Time          Energy          fmax
FIRE:    0 21:39:30    -2934.861011        0.332708
FIRE:    1 21:39:30    -2934.873447        0.332709


/home/mehuldarak/anaconda3/envs/mace/lib/python3.13/site-packages/ase/filters.py:606: RuntimeWarning: logm result may be inaccurate, approximate err = 2.371768235590734e-13
  pos[natoms:] = self.logm(pos[natoms:]) * self.exp_cell_factor


FIRE:    2 21:39:31    -2934.895077        0.332721


/home/mehuldarak/anaconda3/envs/mace/lib/python3.13/site-packages/ase/filters.py:606: RuntimeWarning: logm result may be inaccurate, approximate err = 2.3659783462516224e-13
  pos[natoms:] = self.logm(pos[natoms:]) * self.exp_cell_factor


...

FIRE:  241 21:41:22    -2942.694614        0.073470


/home/mehuldarak/anaconda3/envs/mace/lib/python3.13/site-packages/ase/filters.py:606: RuntimeWarning: logm result may be inaccurate, approximate err = 2.408074168668259e-13
  pos[natoms:] = self.logm(pos[natoms:]) * self.exp_cell_factor


FIRE:  242 21:41:22    -2942.697138        0.080844


/home/mehuldarak/anaconda3/envs/mace/lib/python3.13/site-packages/ase/filters.py:606: RuntimeWarning: logm result may be inaccurate, approximate err = 2.467125110721083e-13
  pos[natoms:] = self.logm(pos[natoms:]) * self.exp_cell_factor


FIRE:  243 21:41:22    -2942.725256        0.039957


In [ ]:
# Save relaxed structure
relaxed_structure = result["final_structure"]
relaxed_structure.to(filename=output_path)

print(f"✅ Relaxed structure saved to: {output_path}")

✅ Relaxed structure saved to: /home/mehuldarak/summer/relax_final/cellrelaxed_LLZO_011_La_code71_sto__Li_100_slab_heavy.cif


/home/mehuldarak/anaconda3/envs/mace/lib/python3.13/site-packages/pymatgen/io/cif.py:1724: FutureWarning: We strongly discourage using implicit binary/text `mode`, and this would not be allowed after 2025-06-01. I.e. you should pass t/b in `mode`.
  with zopen(filename, mode=mode) as file:


: 